In [41]:
import torch 
import numpy as np
import pandas as pd

In [42]:
import torch.nn as nn
import torch.nn.functional as F

In [43]:
df = pd.read_csv("income.csv")

In [44]:
df.head()

,age,sex,education,education-num,marital-status,workclass,occupation,hours-per-week,income,label
0,27,Male,HS-grad,9,Never-married,Private,Craft-repair,40,<=50K,0
1,47,Male,Masters,14,Married,Local-gov,Exec-managerial,50,>50K,1
2,59,Male,HS-grad,9,Divorced,Self-emp,Prof-specialty,20,<=50K,0
3,38,Female,Prof-school,15,Never-married,Federal-gov,Prof-specialty,57,>50K,1
4,64,Female,11th,7,Widowed,Private,Farming-fishing,40,<=50K,0


In [45]:
dummy = pd.get_dummies(df["sex"])

In [46]:
df = pd.concat([df,dummy],axis = 1)

In [47]:
dummy = pd.get_dummies(df["education"])

In [48]:
df = pd.concat([df,dummy],axis = 1)

In [49]:
dummy = pd.get_dummies(df["marital-status"])
df = pd.concat([df,dummy],axis = 1)

In [50]:
dummy = pd.get_dummies(df["workclass"])
df = pd.concat([df,dummy],axis = 1)

In [51]:
dummy = pd.get_dummies(df["occupation"])
df = pd.concat([df,dummy],axis = 1)

In [52]:
y = df["label"].values

In [53]:
df = df.drop(["sex","education","marital-status","workclass","occupation","label"],axis = 1)

In [54]:
df = df.drop(["income"],axis = 1)

In [55]:
X = df.values

In [61]:
X.shape

(30000, 42)

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [60]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [62]:
class Model(nn.Module):
    def __init__(self, in_features=42, h1=100, h2=100, out_features=2):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [63]:
model = Model()

In [64]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [65]:
epochs = 100

for i in range(epochs):
    i+=1
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  1  loss: 0.77015418
epoch: 11  loss: 0.63448018
epoch: 21  loss: 0.50856912
epoch: 31  loss: 0.33515453
epoch: 41  loss: 0.28747404
epoch: 51  loss: 0.27172565
epoch: 61  loss: 0.26245216
epoch: 71  loss: 0.25756511
epoch: 81  loss: 0.25422415
epoch: 91  loss: 0.25444490


In [66]:
# TO EVALUATE THE ENTIRE TEST SET
with torch.no_grad():
    y_val = model.forward(X_test)
    loss = criterion(y_val, y_test)
print(f'{loss:.8f}')

0.25181228
